## Isolation Forest

This is a demo to show my implementation of isolation forest based on [this paper](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf)

> F. T. Liu, K. M. Ting and Z. Zhou, "Isolation Forest," 2008 Eighth IEEE International Conference on Data Mining, Pisa, 2008, pp. 413-422.

Isolation forests are used for anamoly detection and the idea is based on the observation that anomalies have distinctive quantitative properties:

1. they are the minority consisting of fewer instances
2. they have attribute-values that are very different from those of normal instances

In [2]:
import numpy as np

In [9]:
class TreeNode(object):
    def __init__(self, left, right, parent, cutoff_id, cutoff_val, prediction):
        self.left = left
        self.right = right
        self.parent = parent
        self.cutoff_id = cutoff_id
        self.cutoff_val = cutoff_val
        self.prediction = prediction

In [10]:
def split(xTr,yTr):
    N,D = xTr.shape
    assert D > 0 # must have at least one dimension
    assert N > 1 # must have at least two samples
    feature = np.inf
    cut = np.inf

    # randomly select an attribute
    feature = np.random.randint(D)
    
    # randomly select a cutoff point from max and min values of attribute
    f_max = np.max(xTr[:, feature])
    f_min = np.max(xTr[:, feature])
    cut = (f_max - f_min) *np.random.random_sample() + f_min
             
    return feature, cut

In [11]:
def tree(xTr,yTr,depth=np.inf):
    n,d = xTr.shape
    
    if depth <= 1 or len(set(yTr)) == 1 or (xTr == xTr[0]).all():
            return TreeNode(None, None, None, None, None, np.sum(yTr)/n)
        
    fid,cut = split(xTr,yTr, w)
    L_idx = xTr[:,fid] <= cut
    R_idx = xTr[:,fid] > cut
    tree_L = tree(xTr[L_idx, :], yTr[L_idx], depth-1)
    tree_R = tree(xTr[R_idx, :], yTr[R_idx], depth-1)
    tree = TreeNode(tree_L, tree_R, None, fid, cut, np.sum(yTr)/n)
    tree_L.parent = tree
    tree_R.parent = tree
    
    return tree

In [12]:
def forest(xTr, yTr, m, maxdepth=np.inf):
    n, d = xTr.shape
    trees = []
    indices = np.arange(n)
    
    for i in range(m):
        idx = np.random.choice(indices, n, replace=True)
        trees.append(cart(xTr[idx,:],yTr[idx],depth=maxdepth))
    return trees

In [17]:
def c(n):
    return 2.0*(np.log(n-1)+0.5772156649)-(2*(n-1.0)/n)

In [25]:
def size(node):
    if node is None:
        return 0
    else:
        return max(size(node.left), size(node.right)) + 1

In [13]:
def pathLength(x, tree, e):
    #if tree is an external node then return e + c(T.size)
    if node.cutoff_id is None:
        return e + c(size(tree))
    else:
        if x[tree.cutoff_id] < tree.cutoff_val:
            return pathLength(x, tree.left, e + 1)
        else:
            return pathLength(x, T.right, e + 1)